In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import random
import string

In [3]:
df1 = pd.read_csv('/content/drive/MyDrive/DS204/Dataset/X(text).csv')
df2 = pd.read_csv('/content/drive/MyDrive/DS204/Dataset/y(label).csv')

In [4]:
df1

,text
0,"In other words , when there is sufficient evid..."
1,System administrators from the Texas Advanced ...
2,We use a complex four - class classification p...
3,"Lexique Pro , an excellent free tool by SIL In..."
4,"However , the Manipuri news is available only ..."
...,...
3329,The data consisted of 10 classifications ( the...
3330,"However , Google provides researchers with a p..."
3331,"Briefly , total RNA was extracted from the 18 ..."
3332,The Web provides access to large - scale sets ...


In [5]:
df2

,0,1,2
0,Supplement,Document,Produce
1,Supplement,License,Other
2,Material,Data,Use
3,Method,Tool,Use
4,Material,Data,Use
...,...,...,...
3329,Supplement,Document,Produce
3330,Supplement,Website,Use
3331,Method,Tool,Use
3332,Supplement,Website,Introduce


In [6]:
merged_df = pd.concat([df1, df2], axis=1)

In [7]:
merged_df

,text,0,1,2
0,"In other words , when there is sufficient evid...",Supplement,Document,Produce
1,System administrators from the Texas Advanced ...,Supplement,License,Other
2,We use a complex four - class classification p...,Material,Data,Use
3,"Lexique Pro , an excellent free tool by SIL In...",Method,Tool,Use
4,"However , the Manipuri news is available only ...",Material,Data,Use
...,...,...,...,...
3329,The data consisted of 10 classifications ( the...,Supplement,Document,Produce
3330,"However , Google provides researchers with a p...",Supplement,Website,Use
3331,"Briefly , total RNA was extracted from the 18 ...",Method,Tool,Use
3332,The Web provides access to large - scale sets ...,Supplement,Website,Introduce


In [8]:
merged_df['Tên Website'] = [random.choice(string.ascii_letters) for _ in range(len(merged_df))]
merged_df['URL Website'] = [random.choice(string.ascii_letters) for _ in range(len(merged_df))]

In [9]:
merged_df

,text,0,1,2,Tên Website,URL Website
0,"In other words , when there is sufficient evid...",Supplement,Document,Produce,a,O
1,System administrators from the Texas Advanced ...,Supplement,License,Other,u,q
2,We use a complex four - class classification p...,Material,Data,Use,u,G
3,"Lexique Pro , an excellent free tool by SIL In...",Method,Tool,Use,m,w
4,"However , the Manipuri news is available only ...",Material,Data,Use,W,X
...,...,...,...,...,...,...
3329,The data consisted of 10 classifications ( the...,Supplement,Document,Produce,V,S
3330,"However , Google provides researchers with a p...",Supplement,Website,Use,C,V
3331,"Briefly , total RNA was extracted from the 18 ...",Method,Tool,Use,y,k
3332,The Web provides access to large - scale sets ...,Supplement,Website,Introduce,o,E


In [ ]:
#merged_df.to_csv('dataset.csv', index=False)

In [11]:
merged_df['Combined_Labels'] = merged_df.iloc[:, 1:4].apply(lambda x: ','.join(sorted(x)), axis=1)

In [12]:
label_counts = merged_df['Combined_Labels'].value_counts()

In [14]:
label_counts

Data,Material,Use                723
Method,Tool,Use                  437
Code,Method,Produce              246
Document,Produce,Supplement      203
Introduce,Paper,Supplement       170
Code,Method,Use                  148
Data,Introduce,Material          126
Supplement,Use,Website           122
Introduce,Supplement,Website     121
Data,Material,Produce            120
License,Other,Supplement         111
Introduce,Method,Tool            101
Produce,Supplement,Website        84
Algorithm,Method,Use              81
Document,Introduce,Supplement     66
Method,Produce,Tool               47
Media,Produce,Supplement          46
Algorithm,Introduce,Method        45
Algorithm,Compare,Method          37
Algorithm,Method,Produce          33
Document,Supplement,Use           29
Extent,Paper,Supplement           27
Compare,Paper,Supplement          24
Data,Extent,Material              23
Algorithm,Extent,Method           20
Extent,Method,Tool                17
Extent,Supplement,Website         16
C

# **Recommendation dựa theo so sánh giống nhau ở các labels.**

In [ ]:
# Read data from the CSV file
data = merged_df

# Function to build the recommendation system
def build_recommendation_system(labels, data):
    recommendations = []
    
    for index, row in data.iterrows():
        row_labels = [row['0'], row['1'], row['2']]
        if set(labels) == set(row_labels):
            recommendations.append(row['result'])
    
    return recommendations

# Example labels from the classification model
classification_labels = ["Supplement", "Document", "Produce"]

# Build the recommendation system and get the output
recommendations = build_recommendation_system(classification_labels, data)

# Print the recommendations
print("Recommendations:")
for recommendation in recommendations:
    print(recommendation)


# **Content-based Recommendation System**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Load the dataset
df = merged_df

# Extract relevant columns
data = df[['text', '0', '1', '2', 'rate', 'result']]  # Replace 0, 1, 2 with column names if available

# Preprocess text data and extract features
vectorizer = TfidfVectorizer()
text_features = vectorizer.fit_transform(data['text'])

# Calculate similarity
labels = ['Supplement', 'Document', 'Produce']  # Labels from classification task
similarity_scores = cosine_similarity(text_features, text_features)

# Function to get recommended results based on labels and rate
def get_recommendations(labels, similarity_scores, data, top_n=5):
    recommendations = []
    for label in labels:
        label_indices = data[(data['0'] == label) | (data['1'] == label) | (data['2'] == label)].index
        label_scores = similarity_scores[label_indices].sum(axis=0)
        top_indices = label_scores.argsort()[::-1][:top_n]
        top_results = data.loc[top_indices, 'result'].tolist()
        recommendations.extend(top_results)
    return list(set(recommendations))[:top_n]  # Select unique values and limit to top_n

# Get recommended results
recommendations = get_recommendations(labels, similarity_scores, data, top_n=5)

# Print the recommended results
print("Recommended results:")
for i, result in enumerate(recommendations, 1):
    print(f"{i}. {result}")